In [68]:
import pandas as pd
import numpy as np
import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble  import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())

In [69]:
# Definindo o caminho dos dados
data_company_path = "dados/Company.csv"
data_tweet_path = "dados/Tweet.csv"
data_company_tweet_path = "dados/Company_Tweet.csv"

In [70]:
# Ler e colocar em um dataframe
data_company = pd.read_csv(data_company_path)
data_tweet = pd.read_csv(data_tweet_path)
data_company_tweet = pd.read_csv(data_company_tweet_path)

In [71]:
print(data_company_tweet.head())
print(data_company_tweet.shape)

             tweet_id ticker_symbol
0  550803612197457920          AAPL
1  550803610825928706          AAPL
2  550803225113157632          AAPL
3  550802957370159104          AAPL
4  550802855129382912          AAPL
(4336445, 2)


In [72]:
print(data_company)
print(data_company.shape)

  ticker_symbol company_name
0          AAPL        apple
1          GOOG   Google Inc
2         GOOGL   Google Inc
3          AMZN   Amazon.com
4          TSLA    Tesla Inc
5          MSFT    Microsoft
(6, 2)


In [74]:
print(data_tweet.head())
print(data_tweet.shape)

             tweet_id           writer   post_date  \
0  550441509175443456  VisualStockRSRC  1420070457   
1  550441672312512512      KeralaGuy77  1420070496   
2  550441732014223360      DozenStocks  1420070510   
3  550442977802207232     ShowDreamCar  1420070807   
4  550443807834402816     i_Know_First  1420071005   

                                                body  comment_num  \
0  lx21 made $10,008  on $AAPL -Check it out! htt...            0   
1  Insanity of today weirdo massive selling. $aap...            0   
2  S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...            0   
3  $GM $TSLA: Volkswagen Pushes 2014 Record Recal...            0   
4  Swing Trading: Up To 8.91% Return In 14 Days h...            0   

   retweet_num  like_num  
0            0         1  
1            0         0  
2            0         0  
3            0         1  
4            0         1  
(3717964, 7)


In [75]:
dados_merge1 = pd.merge(data_company_tweet,data_company, on='ticker_symbol')
dados_merge1 = dados_merge1.drop(columns=['ticker_symbol'])
dados_merge1.head()

,tweet_id,company_name
0,550803612197457920,apple
1,550803610825928706,apple
2,550803225113157632,apple
3,550802957370159104,apple
4,550802855129382912,apple


In [76]:
dados_tweet_company = pd.merge(dados_merge1,data_tweet,on='tweet_id')
dados_tweet_company = dados_tweet_company.drop(columns=['tweet_id'])
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1


In [77]:
dados_tweet_company = dados_tweet_company.iloc[:500000]
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1


In [78]:
dados_tweet_company.shape

(500000, 7)

## Ajustar os Dados

In [79]:
dados_tweet_company.post_date[0]

1420156789

In [80]:
ts_epoch = dados_tweet_company.post_date[0]
ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
ts

'2015-01-01 20:59:49'

In [81]:
def ajustar_tempo(row):
    ts_epoch = row['post_date']
    ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
    return ts  
new_date = dados_tweet_company.apply(ajustar_tempo,axis=1)
dados_tweet_company['Date'] = list(new_date)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48


In [82]:
dados_tweet_company.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   company_name  500000 non-null  object
 1   writer        494767 non-null  object
 2   post_date     500000 non-null  int64 
 3   body          500000 non-null  object
 4   comment_num   500000 non-null  int64 
 5   retweet_num   500000 non-null  int64 
 6   like_num      500000 non-null  int64 
 7   Date          500000 non-null  object
dtypes: int64(4), object(4)
memory usage: 50.5+ MB


In [83]:
sentenca = dados_tweet_company.body[0]
print(sentenca)

sentenca_2 = dados_tweet_company.body[100]
print(sentenca_2)

#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN $T $IGN $SPY $FB $UGAZ #sentiquant 20150101 19:00:08:298
Our Penny Stock Picks Gained Over 968% In the Past 5 Weeks! Get our next pick early: http://tinyurl.com/p46mnk2 $TRX $AAPL $GOOG


In [84]:
def replace_all(texto):
    palavras_especiais = [word for word in texto.split(' ') if word.startswith(('#','$','@'))]
    #print(palavras_especiais)
    sentenca = ' '.join(x for x in texto.split(' ') if x not in palavras_especiais)
    return sentenca

new_strings = dados_tweet_company.body.apply(replace_all)
dados_tweet_company['Processed_Tweet'] = list(new_strings)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526


In [85]:
'https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664'

'https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664'

In [86]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores(dados_tweet_company.body[0])

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [87]:
dados_tweet_company.shape

(500000, 9)

In [88]:
dados_tweet_company['vader_scores']  = None
dados_tweet_company['vader_scores'] = dados_tweet_company.Processed_Tweet.apply(lambda tweet : sid.polarity_scores(tweet))
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [89]:
# separar os valores do compound - vamos uas-lo para determinar os lbels dos tweets pos/beg
dados_tweet_company['compound']  = dados_tweet_company['vader_scores'].apply(lambda score_dict: score_dict['compound'])
dados_tweet_company.head()


,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0


In [90]:
# definindo os labels usando o compund value como referenecia
def labels(value):
    if value == 0:
        return 0
    elif value > 0:
        return 1
    elif value < 0:
        return -1
    
dados_tweet_company['label']  = dados_tweet_company['compound'].apply(lambda value: labels(value))
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound,label
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0


In [91]:
dados_tweet_company.label.value_counts()

 0    259944
 1    168182
-1     71874
Name: label, dtype: int64

In [92]:
# separando os valores para a machine learning
dados = dados_tweet_company[['Processed_Tweet','label']]
dados.head()

,Processed_Tweet,label
0,20150101 19:00:08:298,0
1,20150101 19:00:08:298,0
2,20150101 19:00:04:526,0
3,20150101 19:00:04:526,0
4,20150101 19:00:04:526,0


In [93]:
datasetVariables_train, datasetVariables_test, datasetTarget_train, datasetTarget_test = train_test_split(dados['Processed_Tweet'], dados['label'], test_size=0.33, random_state=42)

In [94]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(datasetVariables_train)

word_index = tokenizer.word_index
print(len(word_index))

166618


In [95]:
print(word_index)

{'<OOV>': 1, 'http': 2, 'com': 3, 'the': 4, 'apple': 5, 'to': 6, 'in': 7, 'for': 8, 'a': 9, 'it': 10, 'on': 11, 'and': 12, 'is': 13, 'of': 14, 'ly': 15, 'dlvr': 16, 'aapl': 17, 'stock': 18, 'inc': 19, 'p': 20, 'article': 21, '2015': 22, 'bit': 23, 'news': 24, 'i': 25, 'this': 26, 'rt': 27, 'at': 28, 'with': 29, 't': 30, 'from': 31, 'market': 32, 'co': 33, 'watch': 34, 'you': 35, '00': 36, 'up': 37, 'be': 38, 'will': 39, 'buy': 40, 'source': 41, 'stks': 42, 'new': 43, 'x': 44, '…': 45, 'iphone': 46, 'now': 47, 'are': 48, 'us': 49, 'algos': 50, 'seekingalpha': 51, 'sigma': 52, 'lx': 53, 'crossfinder': 54, 'ats': 55, '1': 56, 'https': 57, 'sell': 58, 'more': 59, '10': 60, 'stocks': 61, 'that': 62, 'triggered': 63, 'options': 64, 'my': 65, 'utm': 66, 'as': 67, 'advfn': 68, 'here': 69, 'xref': 70, 'newsalerttweet': 71, 'adw': 72, '1126416…': 73, '2': 74, 'not': 75, 'today': 76, 'me': 77, 'earnings': 78, 'what': 79, 'our': 80, 'has': 81, 'out': 82, 'via': 83, 'price': 84, 'day': 85, 'by': 86

In [96]:
# Transformar as sentencas dos dados de treinamento e teste para sequencias numericas. ex [[1,2,6],[65,952,15,65],[1,2]]
train_data_sentences = tokenizer.texts_to_sequences(datasetVariables_train)
test_data_sentences = tokenizer.texts_to_sequences(datasetVariables_test)

# normalizar o tamnho das sequencias usando padding. ex [[0,1,2,6],[65,952,15,65],[0,0,1,2]]
train_padded = pad_sequences(train_data_sentences, maxlen=300)
test_padded = pad_sequences(test_data_sentences, maxlen=300)

print("Conjunto de treinamento: ","\n",train_padded[0:19])
print("Conjunto de teste: ","\n",test_padded[0:19])

Conjunto de treinamento:  
 [[    0     0     0 ...  3379  3470   150]
 [    0     0     0 ...    23    15 56034]
 [    0     0     0 ...    42    33 35708]
 ...
 [    0     0     0 ...  3891     4  1503]
 [    0     0     0 ...  1961  1153    69]
 [    0     0     0 ...   193   790 23548]]
Conjunto de teste:  
 [[   0    0    0 ... 2305  443  323]
 [   0    0    0 ... 1215  474   45]
 [   0    0    0 ...   41   96  161]
 ...
 [   0    0    0 ...    2  423    3]
 [   0    0    0 ...    9  167  478]
 [   0    0    0 ...  625 1648    1]]


In [97]:
from tensorflow.keras.utils import to_categorical

datasetTarget_train = to_categorical(np.array(datasetTarget_train))
datasetTarget_test = to_categorical(np.array(datasetTarget_test))

In [98]:
from math import sqrt

In [99]:
# first ML model
model_v1 = RandomForestClassifier(random_state=50,criterion='gini')
model_v1.fit(train_padded,datasetTarget_train)
# model_v1 acuracy
model_v1Score = model_v1.score(test_padded,datasetTarget_test)
model_v1Score

0.817260606060606

In [100]:
# save model
import pickle
filename = 'model2_randomForest_80.sav'
pickle.dump(model_v1,open(filename,'wb'))